# Convert pt3 files to Photon-HDF5


# 2. Prepare the data file


Before starting, you have to upload a data file to be converted to Photon-HDF5.
You can use one of our example data files available
[on figshare](https://figshare.com/articles/data_files_for_phconvert/5421565). 


Specify the input data file in the following cell:

In [ ]:
filename = 'data/DNA_FRET_0.5nM.pt3'

In [ ]:
from __future__ import print_function, division
from past.builtins import xrange
from builtins import zip

import os
import struct
import time
from collections import OrderedDict
import numpy as np

In [ ]:
%matplotlib inline
import numpy as np
import phconvert as phc
print('phconvert version: ' + phc.__version__)

In [ ]:
import os
try: 
    with open(filename): pass
    print('Data file found, you can proceed.')
except IOError:
    print('ATTENTION: Data file not found, please check the filename.\n'
          '           (current value "%s")' % filename)

## Load Data

In [ ]:
d, meta = phc.loader.nsalex_pt3(filename,
                                donor = 1,
                                acceptor = 0,
                                alex_period_donor = (0, 2000),
                                alex_period_acceptor = (2000, 3200),
                                excitation_wavelengths = (470e-9, 635e-9),
                                detection_wavelengths = (525e-9, 690e-9),
                                )

In [ ]:
phc.plotter.alternation_hist_usalex(d)

In [ ]:
detectors = d['photon_data']['detectors']

print("Detector    Counts")
print("--------   --------")
for det, count in zip(*np.unique(detectors, return_counts=True)):
    print("%8d   %8d" % (det, count))

## Removing the overflow counts

In [ ]:
nanotimes = d['photon_data']['nanotimes']
detectors = d['photon_data']['detectors']
timestamps = d['photon_data']['timestamps']

overflow_nanotimes = d['photon_data']['nanotimes'] != 0

detectors = detectors[overflow_nanotimes]
timestamps = timestamps[overflow_nanotimes]
nanotimes = nanotimes[overflow_nanotimes]

In [ ]:
print("Detector    Counts")
print("--------   --------")
for det, count in zip(*np.unique(detectors, return_counts=True)):
    print("%8d   %8d" % (det, count))

In [ ]:
d['photon_data']['nanotimes'] = nanotimes
d['photon_data']['detectors'] = detectors
d['photon_data']['timestamps'] = timestamps

## Metadata

In [ ]:
author = 'Biswajit'
author_affiliation = 'Leiden University'
description = 'A demonstrative pt3 data readin.'
sample_name = 'Copper Azurin in 1mM Ascorbate'
dye_names = 'ATTO655'
buffer_name = 'HEPES pH7 with 100 mM NaCl'

### Add meta data

In [ ]:
d['description'] = description

d['sample'] = dict(
    sample_name=sample_name,
    dye_names=dye_names,
    buffer_name=buffer_name,
    num_dyes = len(dye_names.split(',')))

d['identity'] = dict(
    author=author,
    author_affiliation=author_affiliation)

In [ ]:
# Remove some empty groups that may cause errors on saving
_ = meta.pop('dispcurve', None)
_ = meta.pop('imghdr', None)

In [ ]:
d['user'] = {'picoquant': meta}

## Save to Photon-HDF5

In [ ]:
phc.hdf5.save_photon_hdf5(d, overwrite=True)

## Load Photon-HDF5

In [ ]:
from pprint import pprint

In [ ]:
filename = d['_data_file'].filename

In [ ]:
h5data = phc.hdf5.load_photon_hdf5(filename)

In [ ]:
phc.hdf5.dict_from_group(h5data.identity)

In [ ]:
phc.hdf5.dict_from_group(h5data.setup)

In [ ]:
pprint(phc.hdf5.dict_from_group(h5data.photon_data))

In [ ]:
h5data._v_file.close()